# `DISTINCT` unique

Using the syntax `SELECT DISTINCT ...` you can get unique values in any column.

In the next cell, I create everything I need for the examples in this page.

In [1]:
%%bash
docker run --rm -d\
    -e POSTGRES_PASSWORD=postgres \
    --name sql_DISTINCT_examples \
    postgres:15.4 &> /dev/null
sleep 5
docker exec -i sql_DISTINCT_examples psql -U postgres -d postgres

\set QUIET on
CREATE TABLE non_unique_values(
    col1 TEXT,
    col2 TEXT
);
INSERT INTO non_unique_values(col1, col2) VALUES
('A', 'X'),
('A', 'Y'),
('A', 'X'),
('B', 'X'),
('C', 'Y'),
('C', 'Y');
\q

The next cell shows the table I will use for examples in this section.

In [18]:
from IPython.display import HTML
display(HTML("<b style=\"font-size:120%\">=====Test table=====</b>"))
!docker exec sql_DISTINCT_examples psql -U postgres -d postgres -c\
"\
    SELECT * FROM non_unique_values;\
"

 col1 | col2 
------+------
 A    | X
 A    | Y
 A    | X
 B    | X
 C    | Y
 C    | Y
(6 rows)



**Note** don't forget to stop the container when you finish playing with examples.

In [2]:
!docker stop sql_DISTINCT_examples &> /dev/null

## Specific column

If you use only one column as the result of a `SELECT`, the unique values of that column will be retrieved.

In [3]:
%%bash
docker exec -i sql_DISTINCT_examples psql -U postgres -d postgres

SELECT DISTINCT col1 FROM non_unique_values;

\q

 col1 
------
 B
 C
 A
(3 rows)



## Columns combination

If you use multiple columns as the result of a SELECT, you will get every possible combination of values in the columns once.

In [5]:
display(HTML("<b style=\"font-size:120%\">=====DISTINCT col1, col2=====</b>"))
!docker exec sql_DISTINCT_examples psql -U postgres -d postgres -c\
"\
    SELECT DISTINCT col1, col2 FROM non_unique_values;\
"

 col1 | col2 
------+------
 B    | X
 A    | Y
 A    | X
 C    | Y
(4 rows)

